In [7]:
data = spark.read.format("libsvm").load('swift2d://OReilly.os_eea0fae16ed84b69a7875db7dcc2ba81_configs/sample_libsvm_data.txt')


In [5]:

import ibmos2spark

# @hidden_cell
credentials = {
    'auth_url': 'https://identity.open.softlayer.com',
    'project_id': 'df584add39774a4492e9ac43bbfe2944',
    'region': 'dallas',
    'user_id': '27783e2082124d2e9bd2fe45de4ec98d',
    'username': 'member_6159c7d53ab652ef6d55e41ec4e92bf82ce9db34',
    'password': 'W4x3?,k_2WCn_I-1'
}

configuration_name = 'os_eea0fae16ed84b69a7875db7dcc2ba81_configs'
bmos = ibmos2spark.bluemix(sc, credentials, configuration_name)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Please read the documentation of PySpark to learn more about the possibilities to load data files.
# PySpark documentation: https://spark.apache.org/docs/2.0.1/api/python/pyspark.sql.html#pyspark.sql.SparkSession
# The SparkSession object is already initalized for you.
# The following variable contains the path to your file on your Object Storage.
path_2 = bmos.url('OReilly', 'sample_libsvm_data.txt')


In [6]:
path_1

'swift2d://OReilly.os_eea0fae16ed84b69a7875db7dcc2ba81_configs/sample_libsvm_data.txt'

In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [10]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "indexedLabel", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print "Test Error = %g " % (1.0 - accuracy)

treeModel = model.stages[2]
# summary only
print treeModel

+----------+------------+--------------------+
|prediction|indexedLabel|            features|
+----------+------------+--------------------+
|       1.0|         1.0|(692,[121,122,123...|
|       1.0|         1.0|(692,[123,124,125...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
|       1.0|         1.0|(692,[124,125,126...|
+----------+------------+--------------------+
only showing top 5 rows

Test Error = 0 
DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4d62af330aadd04f5cdd) of depth 2 with 5 nodes


In [12]:
treeModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4d62af330aadd04f5cdd) of depth 2 with 5 nodes